In [1]:
import pandas as pd 
import numpy as np
import sqlite3
import pickle
pd.set_option('display.max_columns', None)
from functools import reduce 

In [2]:
conn = sqlite3.connect('database.db')

In [3]:

query = '''
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2012
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2013
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2014
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2015
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2016
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2017
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2018
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2019
    GROUP BY "Month", "LSOA code"
    '''
dataframe2 = pd.read_sql_query(query, conn)

In [4]:
#Train
crime_per_district = dataframe2.copy()
crime_per_district["Falls within"] = crime_per_district["Falls within"].str.rsplit(' ',1).str[0]
crime_per_district = crime_per_district.rename(columns={'LSOA name': 'name', 'LSOA code': 'Geo code', 'Falls within':'Force Name'})

In [5]:
del dataframe2

# Police

In [6]:
df_police = pd.read_excel('Number of police officers per district per year.ods', engine='odf')
df_police = df_police.rename(columns={'As at 31 March...': 'year', 'Geo code': 'Geo code police'})

In [7]:
df_police2 = df_police[df_police['year']>2011]
df_police3 = df_police2[df_police2['year']<2020]
df_police4 = df_police3[df_police3['Worker type']=='Police Officer']
df_police_final = df_police4.groupby(['year','Force Name', 'Geo code police'])[['Total (headcount)']].sum()

In [8]:
del df_police
del df_police2
del df_police3
del df_police4

# Population

In [9]:
df_2012 = pd.read_excel('people2012.xlsx')
df_2013 = pd.read_excel('people2013.xlsx')
df_2014 = pd.read_excel('people2014.xlsx')
df_2015 = pd.read_excel('people2015.xlsx')
df_2016 = pd.read_excel('people2016.xlsx')
df_2017 = pd.read_excel('people2017.xlsx')
df_2018 = pd.read_excel('people2018.xlsx')
df_2019 = pd.read_excel('people2019.xlsx')

C:\Users\blaze\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [10]:
df_2012 = df_2012[['Area Codes', 'All Ages']]
df_2013 = df_2013[['Area Codes', 'All Ages']]
df_2014 = df_2014[['Area Codes', 'All Ages']]
df_2015 = df_2015[['Area Codes', 'All Ages']]
df_2016 = df_2016[['Area Codes', 'All Ages']]
df_2017 = df_2017[['Area Codes', 'All Ages']]
df_2018 = df_2018[['Area Codes', 'All Ages']]
df_2019 = df_2019[['LSOA Code', 'All Ages']]
df_2012 = df_2012.rename(columns={'All Ages':'Population 2012'})
df_2013 = df_2013.rename(columns={'All Ages':'Population 2013'})
df_2014 = df_2014.rename(columns={'All Ages':'Population 2014'})
df_2015 = df_2015.rename(columns={'All Ages':'Population 2015'})
df_2016 = df_2016.rename(columns={'All Ages':'Population 2016'})
df_2017 = df_2017.rename(columns={'All Ages':'Population 2017'})
df_2018 = df_2018.rename(columns={'All Ages':'Population 2018'})
df_2019 = df_2019.rename(columns={'LSOA Code': 'Area Codes', 'All Ages':'Population 2019'})

In [11]:
data_frames = [df_2012, df_2013,df_2014,df_2015,df_2016,df_2017,df_2018,df_2019]

In [12]:
df_pop_merged =  reduce(lambda  left,right: pd.merge(left,right,on=['Area Codes'], how='outer'), data_frames)

In [13]:
del data_frames

In [14]:
del df_2012
del df_2013
del df_2014
del df_2015
del df_2016
del df_2017
del df_2018
del df_2019

In [15]:
df_pop_merged2 = df_pop_merged.copy()
df_pop_merged2 = df_pop_merged2.loc[df_pop_merged.index.repeat(8)]
df_pop_merged2['year']= 2012
df_pop_merged2['LSOA population']= 514261.0
df_pop_merged2.reset_index(inplace=True)

In [16]:
for i in range (1, df_pop_merged2.shape[0]):
    if df_pop_merged2['index'][i] == df_pop_merged2['index'][i-1]:
        df_pop_merged2['year'][i] = df_pop_merged2['year'][i-1]+1

C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/4108131678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_merged2['year'][i] = df_pop_merged2['year'][i-1]+1


In [17]:
for i in range (0, df_pop_merged2.shape[0]):
    if df_pop_merged2['year'][i]==2012:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2012'][i]
    if df_pop_merged2['year'][i]==2013:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2013'][i]
    if df_pop_merged2['year'][i]==2014:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2014'][i]
    if df_pop_merged2['year'][i]==2015:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2015'][i]
    if df_pop_merged2['year'][i]==2016:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2016'][i]
    if df_pop_merged2['year'][i]==2017:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2017'][i]
    if df_pop_merged2['year'][i]==2018:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2018'][i]
    if df_pop_merged2['year'][i]==2019:
        df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2019'][i] 

C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/1683829782.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2012'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/1683829782.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_merged2['LSOA population'][i] = df_pop_merged2['Population 2013'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/1683829782.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [18]:
del df_pop_merged

In [19]:
df_pop_merged2.drop(['Population 2012', 'Population 2013','Population 2014','Population 2015','Population 2016','Population 2017', 'Population 2018', 'Population 2019'], axis=1, inplace=True)

In [20]:
df_pop_merged2 = df_pop_merged2.rename(columns={'Area Codes': 'LSOA code'})

In [21]:
df_population = pd.read_csv("Population of each district.csv")
df_population=df_population.rename(columns={'ladcode21':'Geo code', 'laname21':'name'})

df_districts = pd.read_csv("districts.csv")
df_districts = df_districts.rename(columns={'LAD19CD':'Geo code', 'LAD19NM':'name','PFA19CD': 'Geo code police', 'PFA19NM':'Force Name'})
df_districts.drop(['CSP19CD', 'CSP19NM', 'FID'], axis=1, inplace=True)

df_population2 = df_population.groupby(['name', 'Geo code'])[['population_2012', 'population_2013', 'population_2014', 'population_2015', 'population_2016', 'population_2017', 'population_2018', 'population_2019']].sum()
df_population2.reset_index(inplace=True)
df_population_final = pd.merge(df_population2, df_districts, on='Geo code')
df_population_final = df_population_final.rename(columns={'name_x': 'name'})
df_population_final.drop(['name_y'], axis=1, inplace=True)

In [22]:
del df_population
del df_population2
del df_districts

# Merge population with the police

In [23]:
df_police_final2 = df_police_final.unstack(level=0)
df_police_final2.reset_index(inplace=True)

In [24]:
df_pop_pol = pd.merge(df_police_final2, df_population_final, on='Force Name')
df_pop_pol.drop(('Geo code police', ''), axis=1, inplace=True)

C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/1797237973.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  df_pop_pol = pd.merge(df_police_final2, df_population_final, on='Force Name')


In [25]:
del df_police_final
del df_police_final2

# Add year population and headcount column to merged dataframe

In [26]:
df_pop_pol1 = df_pop_pol.copy()
df_pop_pol1 = df_pop_pol1.loc[df_pop_pol1.index.repeat(8)]
df_pop_pol1['year']= 2012
df_pop_pol1['population']= 61334
df_pop_pol1['headcount'] = 3181
df_pop_pol1.reset_index(inplace=True)

In [27]:
del df_pop_pol

In [28]:
for i in range (1, df_pop_pol1.shape[0]):
    if df_pop_pol1['index'][i] == df_pop_pol1['index'][i-1]:
        df_pop_pol1['year'][i] = df_pop_pol1['year'][i-1]+1

C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/2307358565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_pol1['year'][i] = df_pop_pol1['year'][i-1]+1


In [29]:
for i in range (0, df_pop_pol1.shape[0]):
    if df_pop_pol1['year'][i]==2011:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2011'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2011)][i]
    if df_pop_pol1['year'][i]==2012:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2012'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2012)][i]
    if df_pop_pol1['year'][i]==2013:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2013'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2013)][i]
    if df_pop_pol1['year'][i]==2014:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2014'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2014)][i]
    if df_pop_pol1['year'][i]==2015:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2015'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2015)][i]
    if df_pop_pol1['year'][i]==2016:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2016'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2016)][i]
    if df_pop_pol1['year'][i]==2017:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2017'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2017)][i]
    if df_pop_pol1['year'][i]==2018:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2018'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2018)][i]
    if df_pop_pol1['year'][i]==2019:
        df_pop_pol1['population'][i] = df_pop_pol1['population_2019'][i]
        df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2019)][i]

C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/3806902658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_pol1['population'][i] = df_pop_pol1['population_2012'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/3806902658.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_pol1['headcount'][i] = df_pop_pol1[('Total (headcount)', 2012)][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_4416/3806902658.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [30]:
df_pop_pol1.drop(['population_2012', 'population_2013','population_2014','population_2015','population_2016','population_2017', 'population_2018', 'population_2019'], axis=1, inplace=True)
df_pop_pol1.drop(['index',('Force Name', ''),('Total (headcount)', 2012), ('Total (headcount)', 2013), ('Total (headcount)', 2014), ('Total (headcount)', 2015), ('Total (headcount)', 2016),('Total (headcount)', 2017), ('Total (headcount)', 2018),('Total (headcount)', 2019)], axis=1, inplace=True)


# Add year column to dataframe from sql

In [31]:
crime_per_district['year'] = pd.to_datetime(crime_per_district['Month'], format='%Y-%m').dt.year

# Merge everything together

In [32]:
df_pop_merged2 = df_pop_merged2.rename(columns={'LSOA code':'Geo code'})

In [33]:
data = pd.merge(df_pop_merged2, crime_per_district, on=['year', 'Geo code'])

In [34]:
del df_pop_merged2
del crime_per_district

In [35]:
data.rename(columns={'name': 'LSOA name', 'Geo code':'LSOA code',}, inplace=True)

In [36]:
data.reset_index(inplace=True)

In [37]:
eng1 = pd.read_csv('England1 postcodes.csv')
eng2 = pd.read_csv('England2 postcodes.csv')
eng3 = pd.read_csv('England3 postcodes.csv')
wale = pd.read_csv('Wales postcodes.csv')
scot = pd.read_csv('Scotland postcodes.csv')
ire = pd.read_csv('BT postcodes.csv')
dist_lsoa = pd.concat([eng1, eng2, eng3, wale, scot, ire])

C:\Users\blaze\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18,29,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\blaze\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\blaze\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [38]:
del eng1
del eng2
del eng3
del wale
del scot
del ire

In [39]:
import gc
gc.collect()

118

In [42]:
final_dist_lsoa = dist_lsoa[['District Code', 'LSOA Code']]

In [43]:
final_dist_lsoa.rename(columns={'LSOA Code':'LSOA code', 'District Code':'Geo code'}, inplace=True)

C:\Users\blaze\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [46]:
data3 = pd.merge(df_pop_pol1, final_dist_lsoa, on=['Geo code'])

In [49]:
data3.drop_duplicates(inplace=True)

In [53]:
data2 = pd.merge(data3, data, on=['year', 'LSOA code'])

In [57]:
del data
del data3

In [58]:
data2.drop(['Force Name_y', 'level_0','index'], axis=1, inplace=True)


In [61]:
data2.rename(columns={'Force Name_x':'Force Name'}, inplace=True)

In [63]:
df_IMD = pd.read_csv("IMD_2015_LSOA_score.csv", skiprows = 7)

df_IMD = df_IMD[["Reference area", "2015"]]
df_IMD = df_IMD.rename(columns={"Reference area":"LSOA name", "2015" : "IMD score"})

In [66]:
data_final = pd.merge(data2, df_IMD,  on=['LSOA name']).dropna()
data_final.reset_index(inplace=True)

In [67]:
data_final['prev_month_crimes'] = data_final.groupby(['LSOA name', 'year'])['amount_of_crimes'].shift(1)

In [69]:
data_final.drop(['index'], axis=1, inplace=True)

In [70]:
data_final.to_pickle("data_final.pkl")  